# USING THE Training data generated from a ANN-DQN and trying to predict what action to take based on the image that is 4 images stacked 

In [4]:
# General libraries
import pickle
#For model building
import torch
import torchvision
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
import random
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [5]:
import gym
from gym import wrappers
import torch
import torch.nn as nn
import imageio
import numpy as np
from skimage.transform import resize
import torch.nn.functional as F
import random
import torchvision.transforms as transforms
import tensorflow.compat.v1 as tf

In [6]:
class Atari(object):
    """Wrapper for the environment provided by gym"""
    def __init__(self, envName, no_op_steps=10, agent_history_length=4):
        self.env = gym.make(envName)
        self.state = None
        self.last_lives = 0
        self.no_op_steps = no_op_steps
        self.agent_history_length = agent_history_length

    def reset(self,evaluation=False):

        frame = self.env.reset()
        self.last_lives = 0
        terminal_life_lost = True # Set to true so that the agent starts 
                                  # with a 'FIRE' action when evaluating
        if evaluation:
            for _ in range(random.randint(1, self.no_op_steps)):
                frame, _, _, _ = self.env.step(1) # Action 'Fire'
        processed_frame = frameprocess(frame)   # (★★★)
        self.state = np.repeat(processed_frame, self.agent_history_length, axis=2)
        
        return terminal_life_lost

    def step(self,action):

        new_frame, reward, terminal, info = self.env.step(action)  # (5★)
            
        if info['lives'] < self.last_lives:
            terminal_life_lost = True
        else:
            terminal_life_lost = terminal
        self.last_lives = info['lives']
        
        processed_new_frame = frameprocess(new_frame)   # (6★)
        new_state = np.append(self.state[:, :, 1:], processed_new_frame, axis=2) # (6★)   
        self.state = new_state
        
        return processed_new_frame, reward, terminal, terminal_life_lost, new_frame

# Loads the training data and cleans it

In [7]:
totensor = transforms.ToTensor()

with open('pickeledDataPong', 'rb') as f:
    trainingData = pickle.load(f)
# Reformat training data to predict the next 20 moves
trainset,testset = train_test_split(trainingData,test_size=0.3)
l = len(trainset)
trainimages = []
trainlabels = []
testimages = []
testlabels = []
for i in range(l):
    if trainset[i][1] not in [0,1,2]:
        trainimages.append(totensor(np.array(trainset[i][0])[:,14:66]).cpu().detach().numpy())
        trainlabels.append(np.array([k==trainset[i][1] for k in range(6)]))
lt = len(testset)
for i in range(lt):
    if testset[i][1] not in [0,1,2]:
        testimages.append(totensor(np.array(testset[i][0])[:,14:66]).cpu().detach().numpy())
        testlabels.append(testset[i][1])



train_data = TensorDataset(torch.Tensor(np.array(trainimages)), torch.Tensor(np.array(trainlabels)))
test_data = TensorDataset(torch.Tensor(np.array(testimages)),torch.Tensor(np.array(testlabels)))



FileNotFoundError: [Errno 2] No such file or directory: 'pickeledDataPong'

In [9]:
def frameprocess(frame,frame_height=84, frame_width=65):

    frame_height = frame_height
    frame_width = frame_width
    processed = tf.image.rgb_to_grayscale(frame)
    processed = tf.image.crop_to_bounding_box(processed, 34, 0, 160, 140)
    processed = tf.image.resize_images(processed, 
                                            [frame_height, frame_width], 
                                            method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return processed

In [10]:
def generate_gif(frames_for_gif):

    for idx, frame_idx in enumerate(frames_for_gif): 
        frames_for_gif[idx] = resize(frame_idx, (420, 320, 3), 
                                     preserve_range=True, order=0).astype(np.uint8)
        
    imageio.mimsave("ATARI_PONG.gif", 
                    frames_for_gif, duration=1/30)

In [ ]:
#newTrainingData[0]

In [ ]:
"""# import required libraries
import torch
import torchvision.transforms as T
from PIL import Image

# define a torch tensor
print(trainingData[1000][0].shape)
tensor = newTrainingData[1000][0]

# define a transform to convert a tensor to PIL image
transform = T.ToPILImage()
nd = transforms.ToTensor()
# convert the tensor to PIL image using above transform
img = transform(tensor)
imgten = nd(newTrainingData[1000][0])
print(imgten.shape)
# display the PIL image
img.show()"""

In [12]:
len(train_data)
batch_size = 1

NameError: name 'train_data' is not defined

In [11]:
batch_size = 1

trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,shuffle=True,drop_last=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,shuffle=True,drop_last=True,num_workers=2)


NameError: name 'train_data' is not defined

In [ ]:
""" for image,label in trainloader:
    # define a torch tensor
    print(image[0].shape)
    tensor = image[0]

    # define a transform to convert a tensor to PIL image
    transform = T.ToPILImage()

    # convert the tensor to PIL image using above transform
    img = transform(tensor)

    # display the PIL image
    img.show()
    break"""
    

In [32]:
import torch.nn as nn
import torch.nn.functional as F
batch_size = 1

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(4, 6,8,4)
        self.conv2 = nn.Conv2d(6, 16,4,2)
        self.conv3 = nn.Conv2d(16,32,3,1)
        self.conv4 = nn.Conv2d(32,64,7,1)
        
        self.fc1 = nn.Linear(720*batch_size, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fclast = nn.Linear(84, 6*batch_size)
        self.softmax = torch.nn.Softmax()
        

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        
        x = torch.flatten(x) # flatten all dimensions
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))    
        x =self.fclast(x)    
        sig_out = x.view(batch_size, -1)
        #print(x)
        sig_out = self.softmax(sig_out)
        #print("sig x",sig_out)
        #print(sig_out)
        return sig_out


net = Net()

In [33]:
#%pip install torchinfo 
import torchsummary
batch_size = 16
torchsummary.summary(net)

Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            1,542
├─Conv2d: 1-2                            1,552
├─Conv2d: 1-3                            4,640
├─Conv2d: 1-4                            100,416
├─Linear: 1-5                            86,520
├─Linear: 1-6                            10,164
├─Linear: 1-7                            510
├─Softmax: 1-8                           --
Total params: 205,344
Trainable params: 205,344
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            1,542
├─Conv2d: 1-2                            1,552
├─Conv2d: 1-3                            4,640
├─Conv2d: 1-4                            100,416
├─Linear: 1-5                            86,520
├─Linear: 1-6                            10,164
├─Linear: 1-7                            510
├─Softmax: 1-8                           --
Total params: 205,344
Trainable params: 205,344
Non-trainable params: 0

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# zero the parameter gradients
optimizer.zero_grad()
# forward + backward + optimize
#print(torch.tensor([input]).size())
for epoch in range(1):
    for image,move in trainloader:
        outputs = net(image)
        #print(torch.tensor(outputs))
        #print(move)
        #print(torch.argmax(outputs),torch.tensor(move))
        loss = criterion(outputs,torch.tensor(move))
        loss.backward()
        optimizer.step()

In [ ]:
"""
# zero the parameter gradients
optimizer.zero_grad()

for input,labels in trainloader:
    # forward + backward + optimize
    #print(torch.tensor([input]).size())
    outputs = net(torch.tensor(input))
    loss = sum([criterion(output, l) for (l, output) in zip(outputs, labels)])
    loss.backward()
    optimizer.step()
"""

In [ ]:
def check_accuracy(outputs,target):
    #print(outputs)
    return np.equal(np.array([torch.argmax(outputs)]),np.array(target))

    
    

In [ ]:
accuracy = []
for image,moves in testloader:
    outputs = net(image)
    #print(check_accuracy(outputs,moves))
    accuracy.append(check_accuracy(outputs,moves))
    #print(torch.argmax(outputs))

print(np.mean(accuracy))

In [ ]:
torch.save(net.state_dict(), "savedmodel")

In [ ]:
torch.save(net, "FULLMODEL")

In [ ]:
#2up 3 down 4 up 5 down

In [ ]:
#tt = transforms.ToTensor()
env = Atari("PongDeterministic-v4")
env.reset()
frames=[]
total_reward=0
done = False
env.reset()
i=0

for i in range(300):
    action = 2
    observation, reward, done, info,new_frame = env.step(action)
    frames.append(new_frame)
    env.env.render()
#transform = transforms.ToPILImage()

# convert the tensor to PIL image using above transform
#img = transform(tt(env.state))

# display the PIL image
#img.save("PIL.png")
    
while not done:
    i+=1
    #s = env.state
    action = net(torch.Tensor(np.array([totensor(env.state).cpu().detach().numpy() for i in range(batch_size)])))
    #print("action",action)
    #env.step(action)
    #print(torch.argmax(action))
    observation, reward, done, info,new_frame = env.step(int(torch.argmax(action[0])))
    #(env.state).show()
    
    frames.append(new_frame)
    env.env.render()

print(i)
env.reset()
generate_gif(frames)

# Idea Abandoned As 40% accuracy seems too bad
# Was planning on converting to SNN but not worth it as some accuarcy will be lost in the process
 